# Calculo dos Resultados dos Clientes Conta Única

## 0 - Setup  

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:.3f}')
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows',50)


import numpy as np 
from matplotlib import pyplot as plt

%pip install -r requirements.txt
from google.cloud import bigquery
data_origin = "`lemoncake-prod.playground.dados_jaime_analise_conta_unica`"
project_id = 'lemoncake-prod'
client = bigquery.Client(project = project_id)

%pip install plotly==5.17.0
import plotly.graph_objects as go
import plotly.express as px

from datetime import datetime, timedelta

ERROR: Ignored the following versions that require a different python version: 2.10.0 Requires-Python >=3.6, <3.10; 2.11.0 Requires-Python >=3.6, <3.10; 2.12.0 Requires-Python >=3.6, <3.10; 2.13.0 Requires-Python >=3.6, <3.10; 2.13.1 Requires-Python >=3.6, <3.10; 2.14.0 Requires-Python >=3.6, <3.10; 2.15.0 Requires-Python >=3.6, <3.10; 2.16.0 Requires-Python >=3.6, <3.10; 2.16.1 Requires-Python >=3.6, <3.10; 2.17.0 Requires-Python >=3.6, <3.10; 2.18.0 Requires-Python >=3.6, <3.10; 2.19.0 Requires-Python >=3.6, <3.10; 2.20.0 Requires-Python >=3.6, <3.10; 2.21.0 Requires-Python >=3.6, <3.10; 2.22.0 Requires-Python >=3.6, <3.10; 2.22.1 Requires-Python >=3.6, <3.10; 2.23.0 Requires-Python >=3.6, <3.10; 2.23.1 Requires-Python >=3.6, <3.10; 2.23.2 Requires-Python >=3.6, <3.10; 2.23.3 Requires-Python >=3.6, <3.10; 2.24.0 Requires-Python >=3.6, <3.10; 2.24.1 Requires-Python >=3.6, <3.10; 2.25.0 Requires-Python >=3.6, <3.10; 2.25.1 Requires-Python >=3.6, <3.10; 2.25.2 Requires-Python >=3.6, <3.

Note: you may need to restart the kernel to use updated packages.


c:\Users\DELL\anaconda3\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Note: you may need to restart the kernel to use updated packages.


In [71]:
data_inicio_teste = '2023-11-08'
current_date = datetime.now().date()

## 1 - Lendo os dados 

In [72]:
clientes_teste_filename = 'clientes_teste.csv'
clientes_gcontrole_filename = 'clientes_grupo_controle.csv'
folder_name = 'clientes'

teste_path = "\\".join([folder_name,clientes_teste_filename])
gcontrole_path = "\\".join([folder_name,clientes_gcontrole_filename])

clientes_teste_df = pd.read_csv(teste_path)
n_clientes = clientes_teste_df.shape[0]

clientes_gcontrole_raw = pd.read_csv(teste_path)
clientes_gcontrole_df = clientes_gcontrole_raw.sample(n_clientes, random_state=42)

In [73]:
query = f'''SELECT
            *
         FROM {data_origin}'''

data_raw = client.query(query).to_dataframe()
data_raw.shape

(5580, 11)

In [74]:
data_raw['cobranca_vencida'] = data_raw['data_vencimento'] < current_date


In [75]:
def treat_diff(x):
    if x is None:
        result = x
    else:
        result = int(x)
    return result

In [76]:
data_raw['diff_pagamento_vencimento'] = data_raw['diff_pagamento_vencimento'].apply(treat_diff)
data_raw['data_vencimento'] = data_raw['data_vencimento'].apply(pd.to_datetime)

## 2 - Calculo dos resultados

### 2.1 - Taxa de pagamentos em dia da conta única 

In [77]:
def check_pagamento_em_dia(x): 
    if np.isnan(x):
        result = False
    else:
        if x <= 0:
            result = True
        else:
            result = False
    return result 

In [78]:
def get_taxa_pgt_em_dia(df, janela_inicio, janela_fim, tipo):
    cond0 = df['data_envio'] > janela_inicio
    cond1 = df['data_envio'] < janela_fim 
    cond2 = df['data_vencimento'] < janela_fim
    cond3 = df['tipo'] == tipo 

    cond = (((cond0 & cond1) & cond2)) & cond3 
    cobrancas_de_interesse = df.loc[cond,:].copy()
    cobrancas_de_interesse['pagamento_em_dia'] = cobrancas_de_interesse['diff_pagamento_vencimento'].apply(check_pagamento_em_dia)

    grouped = cobrancas_de_interesse[['pagamento_em_dia']].groupby(by='pagamento_em_dia')
    counts = grouped.size()
    
    if cobrancas_de_interesse.shape[0] == 0:
        taxa = 0
    else:
        taxa = counts[True]/counts.sum()
    return taxa


In [79]:
def get_qt_pgt_em_dia(df, janela_inicio, janela_fim, tipo):
    cond0 = df['data_envio'] > janela_inicio
    cond1 = df['data_envio'] < janela_fim 
    cond2 = df['data_vencimento'] < janela_fim
    cond3 = df['tipo'] == tipo 

    cond = (((cond0 & cond1) & cond2)) & cond3 
    cobrancas_de_interesse = df.loc[cond,:].copy()
    cobrancas_de_interesse['pagamento_em_dia'] = cobrancas_de_interesse['diff_pagamento_vencimento'].apply(check_pagamento_em_dia)

    grouped = cobrancas_de_interesse[['pagamento_em_dia']].groupby(by='pagamento_em_dia')
    counts = grouped.size()
    
    if cobrancas_de_interesse.shape[0] == 0:
        qt = 0
    else:
        qt = counts[True]
    return qt, counts.sum(), qt/counts.sum()

In [80]:
time_range = np.arange(np.datetime64(data_inicio_teste), 
                       datetime.strftime(current_date,'%Y-%m-%d'),
                       np.timedelta64(1,'D'))

qt_teste = []
qt_vencidas_teste =[]
txs_teste = []
for date in time_range:
    qt_pgt_em_dia,qt_vencidas,tx = get_qt_pgt_em_dia(df=data_raw,
                                        janela_inicio = data_inicio_teste,
                                        janela_fim = str(date),
                                        tipo = 'teste')
    qt_teste.append(qt_pgt_em_dia)
    qt_vencidas_teste.append(qt_vencidas)
    txs_teste.append(tx)
    
qt_controle = []
qt_vencidas_controle =[]
txs_controle = []
for date in time_range:
    qt_pgt_em_dia,qt_vencidas,tx = get_qt_pgt_em_dia(df=data_raw,
                                        janela_inicio = data_inicio_teste,
                                        janela_fim = str(date),
                                        tipo = 'controle')
    qt_controle.append(qt_pgt_em_dia)
    qt_vencidas_controle.append(qt_vencidas)
    txs_controle.append(tx)

qt_pgt_dia_df = pd.DataFrame({'data':time_range,
                              'qt_pgt_em_dia_teste': qt_teste,
                              'qt_vencidas_teste': qt_vencidas_teste,
                              'taxa_teste': txs_teste,
                              'qt_pgt_em_dia_controle': qt_controle,
                              'qte_vencidas_controle': qt_vencidas_controle,
                              'taxa_controle': txs_controle})


C:\Users\DELL\AppData\Local\Temp\ipykernel_4352\1531623989.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\DELL\AppData\Local\Temp\ipykernel_4352\1531623989.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\DELL\AppData\Local\Temp\ipykernel_4352\1531623989.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\DELL\AppData\Local\Temp\ipykernel_4352\1531623989.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\DELL\AppData\Local\Temp\ipykernel_4352\1531623989.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\DELL\AppData\Local\Temp\ipykernel_4352\1531623989.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\DELL\AppData\Local\Temp\ipykernel_4352\1531623989.py:18: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\DELL\AppData\Local\Temp\ipykernel_4352\1531623989.py:18: RuntimeWarning:

invalid value encountered in

In [81]:
fig = px.line(qt_pgt_dia_df[['data','taxa_teste','taxa_controle']], 
              x = 'data', 
              y = qt_pgt_dia_df[['data','taxa_teste','taxa_controle']].columns)
fig.show()

In [82]:
qt_pgt_dia_df

,data,qt_pgt_em_dia_teste,qt_vencidas_teste,taxa_teste,qt_pgt_em_dia_controle,qte_vencidas_controle,taxa_controle
0,2023-11-08,0,0,NaN,0,0,NaN
1,2023-11-09,0,0,NaN,0,0,NaN
2,2023-11-10,0,0,NaN,0,0,NaN
3,2023-11-11,0,0,NaN,0,0,NaN
4,2023-11-12,0,0,NaN,0,0,NaN
...,...,...,...,...,...,...,...
56,2024-01-03,53,218,0.243,61,184,0.332
57,2024-01-04,56,224,0.250,63,192,0.328
58,2024-01-05,57,226,0.252,67,199,0.337
59,2024-01-06,73,266,0.274,69,206,0.335


## Frequencia e Severidade

In [83]:
def get_frequencia(df, janela_inicio, janela_fim,n,tipo=['teste','controle']):
    cond0 = df['data_envio'] >= janela_inicio
    cond1 = df['data_envio'] <= janela_fim
    cond2 = df['data_vencimento'] <= janela_fim
    cond3 = df['tipo'].isin(tipo)
    cond = ((cond0 & cond1) & cond2) & cond3

    cob_interesse = data_raw.loc[cond,:].copy()
    cob_interesse['pagamento_em_dia'] = cob_interesse['diff_pagamento_vencimento'].apply(check_pagamento_em_dia)

    grouped = cob_interesse[['numero_instalacao','tipo','pagamento_em_dia']].groupby(by=['numero_instalacao','tipo'], as_index= False)
    df = grouped.agg(lambda x: list(x)[-n:])
    df['frequencia'] = df['pagamento_em_dia'].apply(lambda x: 1 - (sum(x)/len(x)))
    return df


In [84]:
def get_corrected_diff(x, data_ref):
    data_vencimento, status,  diff = x
    if status == 'waitingPayment':
        result = np.datetime64(data_ref) - data_vencimento
        result = result.days
    else:
        result = diff    
    return result

def get_severidade(df, janela_inicio, janela_fim,n,tipo=['teste','controle']):
    cond0 = df['data_envio'] >= janela_inicio
    cond1 = df['data_envio'] <= janela_fim
    cond2 = df['data_vencimento'] <= janela_fim
    cond3 = df['tipo'].isin(tipo)
    cond = ((cond0 & cond1) & cond2) & cond3

    cob_interesse = data_raw.loc[cond,:].copy()
    cob_interesse['pagamento_em_dia'] = cob_interesse['diff_pagamento_vencimento'].apply(check_pagamento_em_dia)
    cob_interesse['corrected_diff'] = cob_interesse[['data_vencimento',
                                                     'status',
                                                     'diff_pagamento_vencimento']].apply(get_corrected_diff, data_ref=janela_fim, axis=1)

    grouped = cob_interesse[['numero_instalacao','tipo','corrected_diff']].groupby(by=['numero_instalacao','tipo'], as_index= False)
    df = grouped.agg(lambda x: list(x)[-n:])
    df['severidade'] = df['corrected_diff'].apply(lambda x: np.mean(x))
    return df

### Frequência pré conta única vs. frequência pós conta única:


In [85]:
frequencia_pre_df = get_frequencia(df = data_raw, 
                                   janela_inicio = '20222-01-01',
                                   janela_fim = data_inicio_teste,
                                   n = 3)

grouped  = frequencia_pre_df[['tipo','frequencia']].groupby(by='tipo')
grouped.mean()

,frequencia
tipo,
controle,0.725
teste,0.748


In [86]:
frequencia_pos_df = get_frequencia(df = data_raw, 
                                   janela_inicio = data_inicio_teste,
                                   janela_fim = datetime.strftime(current_date,'%Y-%m-%d'),
                                   n = 4)

frequencia_pos_df
grouped  = frequencia_pos_df[['tipo','frequencia']].groupby(by='tipo')
grouped.mean()

,frequencia
tipo,
controle,0.696
teste,0.726


### Severidade pré vs severidade pós conta única 

In [87]:
severidade_pre_df = get_severidade(df = data_raw, 
                                   janela_inicio = '20222-01-01',
                                   janela_fim = data_inicio_teste,
                                   n = 3)

grouped  = severidade_pre_df[['tipo','severidade']].groupby(by='tipo')
grouped.mean()

,severidade
tipo,
controle,7.530
teste,6.805


In [88]:
severidade_pos_df = get_severidade(df = data_raw, 
                                   janela_inicio = data_inicio_teste,
                                   janela_fim = datetime.strftime(current_date,'%Y-%m-%d'),
                                   n = 4)

grouped  = severidade_pos_df[['tipo','severidade']].groupby(by='tipo')
grouped.mean()

,severidade
tipo,
controle,6.859
teste,6.110
